In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgbxqx
import catboost as cbt
import xgboost as xgb
import time 
from river import stream
from statistics import mode

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('/Users/suchirmvelpanur/Desktop/ISFCR Internship/datasets/IDS Datasets/ALLFLOWMETER_HIKARI2021.csv')

In [5]:
df

,Unnamed: 0.1,Unnamed: 0,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,...,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,traffic_category,Label
0,0,0,Cg61Jch3vdz9DBptj,103.255.15.23,13316,128.199.242.104,443,2.207588,15,14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML,1
1,1,1,CdRIlqLWdj35Y9vW9,103.255.15.23,13318,128.199.242.104,443,15.624266,15,14,...,1.534300e+07,1.534300e+07,1.534300e+07,1.534300e+07,0.0,29200,65160,0,Bruteforce-XML,1
2,2,2,CLzp9Khd0Y09Qkgrg,103.255.15.23,13320,128.199.242.104,443,12.203357,14,13,...,1.196814e+07,1.196814e+07,1.196814e+07,1.196814e+07,0.0,29200,65160,0,Bruteforce-XML,1
3,3,3,Cnf1YA4iLB4CSNWB88,103.255.15.23,13322,128.199.242.104,443,9.992448,14,13,...,9.759205e+06,9.759205e+06,9.759205e+06,9.759205e+06,0.0,29200,65160,0,Bruteforce-XML,1
4,4,4,C4ZKvv3fpO72EAOsJ6,103.255.15.23,13324,128.199.242.104,443,7.780611,14,14,...,7.545305e+06,7.545305e+06,7.545305e+06,7.545305e+06,0.0,29200,65160,0,Bruteforce-XML,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555273,555273,280838,C9b6Aa2csiogu3vVp9,103.255.15.42,138,103.255.15.255,138,0.000000,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0,XMRIGCC CryptoMiner,1
555274,555274,280839,CGDT4r4PAbp3mvaI6k,103.255.15.42,138,103.255.15.255,138,0.000000,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0,XMRIGCC CryptoMiner,1
555275,555275,280840,CJUxTk4Qd0kHliUKR9,103.255.15.42,138,103.255.15.255,138,0.000000,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0,XMRIGCC CryptoMiner,1
555276,555276,280841,CknUJi2R1iYJG3li3k,103.255.15.42,138,103.255.15.255,138,0.000000,1,0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0,0,0,XMRIGCC CryptoMiner,1


In [6]:
df.head(10)

,Unnamed: 0.1,Unnamed: 0,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,...,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,traffic_category,Label
0,0,0,Cg61Jch3vdz9DBptj,103.255.15.23,13316,128.199.242.104,443,2.207588,15,14,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML,1
1,1,1,CdRIlqLWdj35Y9vW9,103.255.15.23,13318,128.199.242.104,443,15.624266,15,14,...,1.534300e+07,1.534300e+07,1.534300e+07,1.534300e+07,0.0,29200,65160,0,Bruteforce-XML,1
2,2,2,CLzp9Khd0Y09Qkgrg,103.255.15.23,13320,128.199.242.104,443,12.203357,14,13,...,1.196814e+07,1.196814e+07,1.196814e+07,1.196814e+07,0.0,29200,65160,0,Bruteforce-XML,1
3,3,3,Cnf1YA4iLB4CSNWB88,103.255.15.23,13322,128.199.242.104,443,9.992448,14,13,...,9.759205e+06,9.759205e+06,9.759205e+06,9.759205e+06,0.0,29200,65160,0,Bruteforce-XML,1
4,4,4,C4ZKvv3fpO72EAOsJ6,103.255.15.23,13324,128.199.242.104,443,7.780611,14,14,...,7.545305e+06,7.545305e+06,7.545305e+06,7.545305e+06,0.0,29200,65160,0,Bruteforce-XML,1
5,5,5,CyC8D5X7IIG7U95l4,103.255.15.23,13326,128.199.242.104,443,4.571433,14,13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML,1
6,6,6,CEXyM013OxRuUddrS2,103.255.15.23,13328,128.199.242.104,443,2.192640,14,13,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,29200,65160,0,Bruteforce-XML,1
7,7,7,CVFc4q26WLSGblwO2c,103.255.15.23,13330,128.199.242.104,443,16.082514,14,14,...,1.584838e+07,1.584838e+07,1.584838e+07,1.584838e+07,0.0,29200,65160,0,Bruteforce-XML,1
8,8,8,CCvZhO2f7ztLs9Hopc,103.255.15.23,13332,128.199.242.104,443,13.873240,15,14,...,1.359359e+07,1.359359e+07,1.359359e+07,1.359359e+07,0.0,29200,65160,0,Bruteforce-XML,1
9,9,9,CIPZU1mfhrkqqix49,103.255.15.23,13334,128.199.242.104,443,11.331464,14,13,...,1.109482e+07,1.109482e+07,1.109482e+07,1.109482e+07,0.0,29200,65160,0,Bruteforce-XML,1


In [7]:
df.tail(2)

,Unnamed: 0.1,Unnamed: 0,uid,originh,originp,responh,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,...,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,traffic_category,Label
555276,555276,280841,CknUJi2R1iYJG3li3k,103.255.15.42,138,103.255.15.255,138,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,XMRIGCC CryptoMiner,1
555277,555277,280842,C82mlb2i3zEzpeTGQk,103.255.15.42,138,103.255.15.255,138,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,XMRIGCC CryptoMiner,1


In [8]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 555278 rows and 88 columns


In [9]:
# checking the data types of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555278 entries, 0 to 555277
Data columns (total 88 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0.1              555278 non-null  int64  
 1   Unnamed: 0                555278 non-null  int64  
 2   uid                       555278 non-null  object 
 3   originh                   555278 non-null  object 
 4   originp                   555278 non-null  int64  
 5   responh                   555278 non-null  object 
 6   responp                   555278 non-null  int64  
 7   flow_duration             555278 non-null  float64
 8   fwd_pkts_tot              555278 non-null  int64  
 9   bwd_pkts_tot              555278 non-null  int64  
 10  fwd_data_pkts_tot         555278 non-null  int64  
 11  bwd_data_pkts_tot         555278 non-null  int64  
 12  fwd_pkts_per_sec          555278 non-null  float64
 13  bwd_pkts_per_sec          555278 non-null  f

In [10]:
# summary statistics
df.describe()

,Unnamed: 0.1,Unnamed: 0,originp,responp,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Label
count,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,555278.000000,...,5.552780e+05,5.552780e+05,5.552780e+05,5.552780e+05,5.552780e+05,5.552780e+05,555278.000000,555278.000000,555278.000000,555278.000000
mean,277638.500000,137369.392132,38673.198034,5082.161647,9.306544,18.264091,18.892578,7.956928,15.803417,1720.704202,...,1.335883e+05,3.490637e+06,5.000630e+06,8.312942e+06,4.073219e+06,7.992576e+05,14750.725718,21005.879513,4546.872210,0.067887
std,160295.095724,99946.804651,18071.825216,12915.812900,101.802730,463.901484,469.078877,73.441224,467.349374,4994.579973,...,8.289045e+06,1.804930e+07,2.326516e+07,9.683372e+07,1.915958e+07,7.206576e+06,15003.491439,29155.406813,11069.456088,0.251552
min,0.000000,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,138819.250000,52201.000000,28581.000000,53.000000,0.000311,1.000000,1.000000,1.000000,0.000000,4.430152,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,277638.500000,121610.500000,42693.500000,443.000000,0.026218,2.000000,2.000000,2.000000,2.000000,39.760205,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5840.000000,0.000000,0.000000,0.000000
75%,416457.750000,211889.750000,53120.000000,443.000000,0.347231,10.000000,12.000000,3.000000,5.000000,80.376058,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,29200.000000,65160.000000,402.000000,0.000000
max,555277.000000,350709.000000,65535.000000,65522.000000,17393.030600,326292.000000,326827.000000,3953.000000,326822.000000,223696.213333,...,5.438463e+09,2.998038e+08,2.998038e+08,1.738680e+10,2.998038e+08,2.029962e+08,65535.000000,65535.000000,65535.000000,1.000000


In [11]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 0.


In [12]:
# checking for the total amount of null values
df.isnull().sum()

Unnamed: 0.1            0
Unnamed: 0              0
uid                     0
originh                 0
originp                 0
                       ..
fwd_init_window_size    0
bwd_init_window_size    0
fwd_last_window_size    0
traffic_category        0
Label                   0
Length: 88, dtype: int64

In [13]:
# => there are no null values or duplicate rows in the dataset

In [14]:
# Find the number of unique values in each column
unique_counts = df.nunique()

print(unique_counts)

Unnamed: 0.1            555278
Unnamed: 0              350710
uid                     555278
originh                   2899
originp                  62886
                         ...  
fwd_init_window_size       515
bwd_init_window_size       185
fwd_last_window_size      3461
traffic_category             6
Label                        2
Length: 88, dtype: int64


In [15]:
value_count = df.value_counts()
value_count

Unnamed: 0.1  Unnamed: 0  uid                 originh         originp  responh          responp  flow_duration  fwd_pkts_tot  bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  bwd_pkts_per_sec  flow_pkts_per_sec  down_up_ratio  fwd_header_size_tot  fwd_header_size_min  fwd_header_size_max  bwd_header_size_tot  bwd_header_size_min  bwd_header_size_max  flow_FIN_flag_count  flow_SYN_flag_count  flow_RST_flag_count  fwd_PSH_flag_count  bwd_PSH_flag_count  flow_ACK_flag_count  fwd_URG_flag_count  bwd_URG_flag_count  flow_CWR_flag_count  flow_ECE_flag_count  fwd_pkts_payload.min  fwd_pkts_payload.max  fwd_pkts_payload.tot  fwd_pkts_payload.avg  fwd_pkts_payload.std  bwd_pkts_payload.min  bwd_pkts_payload.max  bwd_pkts_payload.tot  bwd_pkts_payload.avg  bwd_pkts_payload.std  flow_pkts_payload.min  flow_pkts_payload.max  flow_pkts_payload.tot  flow_pkts_payload.avg  flow_pkts_payload.std  fwd_iat.min  fwd_iat.max   fwd_iat.tot   fwd_iat.avg    fwd_iat.std    bwd_iat.min  b

In [16]:
# Assuming df is your DataFrame
def print_unique_values(df):
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"Column '{column}' has {len(unique_values)} unique values:")
        print(unique_values)
        print("\n")

print_unique_values(df)

Column 'Unnamed: 0.1' has 555278 unique values:
[     0      1      2 ... 555275 555276 555277]


Column 'Unnamed: 0' has 350710 unique values:
[     0      1      2 ... 280840 280841 280842]


Column 'uid' has 555278 unique values:
['Cg61Jch3vdz9DBptj' 'CdRIlqLWdj35Y9vW9' 'CLzp9Khd0Y09Qkgrg' ...
 'CJUxTk4Qd0kHliUKR9' 'CknUJi2R1iYJG3li3k' 'C82mlb2i3zEzpeTGQk']


Column 'originh' has 2899 unique values:
['103.255.15.23' '103.255.15.27' '103.255.15.20' ... '128.199.193.172'
 '192.241.215.105' '205.185.126.174']


Column 'originp' has 62886 unique values:
[13316 13318 13320 ...   990 35021 14711]


Column 'responh' has 7991 unique values:
['128.199.242.104' '128.199.88.81' '103.255.15.23' ... '103.120.247.252'
 '104.18.5.86' '104.18.4.86']


Column 'responp' has 4557 unique values:
[  443 42000 42001 ... 20881  6369 20800]


Column 'flow_duration' has 193558 unique values:
[ 2.207588 15.624266 12.203357 ...  1.525603  1.530101  1.52896 ]


Column 'fwd_pkts_tot' has 1448 unique values:
[  

In [17]:
df['traffic_category'].unique()

array(['Bruteforce-XML', 'Bruteforce', 'Background', 'Benign', 'Probing',
       'XMRIGCC CryptoMiner'], dtype=object)

In [18]:
df['Label'].unique()

array([1, 0])

In [19]:
filtered_label_0 = df.loc[df['Label'] == 0, 'traffic_category']
print(filtered_label_0.unique())


['Background' 'Benign']


In [20]:
filtered_label_0 = df.loc[df['Label'] == 1, 'traffic_category']
print(filtered_label_0.unique())

['Bruteforce-XML' 'Bruteforce' 'Probing' 'XMRIGCC CryptoMiner']


In [21]:
df.Label.value_counts()

Label
0    517582
1     37696
Name: count, dtype: int64

In [22]:
'''
=> Label 0 corresponding to all normal/benign packets
=> Label 1 corresponds to malicious/unauthorized network traffic
'''

'\n=> Label 0 corresponding to all normal/benign packets\n=> Label 1 corresponds to malicious/unauthorized network traffic\n'

In [23]:
# Split into Training and Testing Sets

In [24]:
X = df.drop(['Label'],axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0) #shuffle=False

In [25]:
#SMOTE(Synthetic Minority Over-sampling Technique) to solve class-imbalance

In [26]:
pd.Series(y_train).value_counts()

Label
0    413981
1     30241
Name: count, dtype: int64

In [31]:
from imblearn.over_sampling import SMOTE

# Calculate the ratio of minority to majority class
minority_majority_ratio = 37696 / 517582  # Ratio of label 1 to label 0

# Define the desired ratio after resampling
desired_ratio = 0.2  # For example, aiming for a 1:5 ratio of minority to majority class

# Calculate the number of samples needed for label 1 to achieve the desired ratio
desired_samples_label_1 = int(desired_ratio * 517582)  # 20% of the majority class samples

# Set the sampling strategy
sampling_strategy = {1: desired_samples_label_1}  # Resample label 1 to desired_samples_label_1

# Initialize SMOTE with the chosen sampling strategy
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)

# Apply SMOTE to your dataset
X_resampled, y_resampled = smote.fit_resample(X, y)


ValueError: could not convert string to float: 'Cg61Jch3vdz9DBptj'